In [1]:
import urllib.request
import numpy
from config import config
from bs4 import BeautifulSoup as bs
import requests
import os.path
# config = __import__("config").config

In [2]:
url = config['paths']['dataseturl']
dataset_version = config['dataset_details']['dataset_version']
lang = config['dataset_details']['programming_language']
data_path = config['paths']['datapath']

def download_file(url, parent_path=data_path):
    file = url.split("/")[-1]
    save_path = parent_path + file 
    if os.path.isfile(save_path):
        print("File already exists")
    else:
        #assuming that data folder already exists - otherwise os.makedirs could be used, but could cause problem with permissions
        with open(save_path, 'wb') as f:
            url_content = requests.get(url)
            f.write(url_content.content)
        print("Download finished")

def retrieve_link(url, link_details, download):
    #read the url 
    with urllib.request.urlopen(url) as website:
        url_content = website.read()
    url_content_structured = bs(url_content, 'html.parser')
    #find all links
    all_links = url_content_structured.find_all("a", href=True)
    #find valid links that contain specific version of CIFAR and programming language - by the text near the tag
    valid_links = list(filter(lambda x: all(detail in x.text.lower().split(" ") for detail in link_details), all_links))
    if len(valid_links) ==0:
        print("No links matching your needs could be found")
    else:
        modified_links = []
        for link in valid_links:
            #unfortunately this is very specific and for this website, otherwise I wouldn't split
            current_link = "/".join(url.split("/")[:-1]) + "/" + link.get("href")
            print("Attempting to download %s", current_link)
            if download:
                download_file(current_link)
            else:
                modified_links.append(current_link)
        return modified_links


In [4]:
#long way
###retrieve_link(url, link_details=[dataset_version, lang], download=True)
#or

links = retrieve_link(url, link_details=[dataset_version, lang], download=False)
for link in links:
    download_file(link)

    
#short way
# downloadURL = config['paths']['downloadurl']
# download_file(downloadURL)

Attempting to download %s https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Download finished


In [6]:
import tarfile